# PDF 보고서 인덱싱 및 테스트

> 목표: 멀티모달 RAG 시스템 구축

## 환경 설정

In [1]:
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

True

In [21]:
import os
from glob import glob

from pprint import pprint
import json

import re
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
from langfuse.langchain import CallbackHandler

# 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

In [4]:
import nest_asyncio

# Jupyter 환경에서 비동기 이벤트 루프 중첩 허용
nest_asyncio.apply()

---

## 1. 문서 파싱

### 1\) 문서 분할

> 일부 문서 내용 파싱 오류 발생 -> 10개 페이지 단위로 문서 분할

In [5]:
import fitz
import os
from pathlib import Path
from typing import List

def split_pdf_by_pages(pdf_path: str) -> List[str]:
    """
    PDF를 페이지 단위로 분할
    """
    pages_per_chunk = 10
    
    # 원본 PDF 정보 확인
    doc = fitz.open(pdf_path)
    total_pages = len(doc)
    
    print(f"📄 원본 파일: {os.path.basename(pdf_path)}")
    print(f"📊 총 페이지: {total_pages}")
    print(f"✂️ 분할 단위: {pages_per_chunk}페이지씩")
    
    split_files = []
    base_name = Path(pdf_path).stem
    
    # 페이지 단위로 분할
    for start_page in range(0, total_pages, pages_per_chunk):
        end_page = min(start_page + pages_per_chunk - 1, total_pages - 1)
        
        # 새 문서 생성
        new_doc = fitz.open()
        new_doc.insert_pdf(doc, from_page=start_page, to_page=end_page)
        
        # 파일명 생성
        chunk_filename = f"{base_name}_chunk_{start_page+1:03d}-{end_page+1:03d}.pdf"
        output_dir = os.path.join(os.path.dirname(pdf_path), 'split_pdfs')
        os.makedirs(output_dir, exist_ok=True)
        chunk_path = os.path.join(output_dir, chunk_filename)
        
        # 저장
        new_doc.save(chunk_path)
        new_doc.close()
        
        split_files.append(chunk_path)
        print(f"✅ 생성: {chunk_filename} (페이지 {start_page+1}-{end_page+1})")
    
    doc.close()
    print(f"🎉 분할 완료: {len(split_files)}개 파일\n")
    
    return split_files

In [6]:
# 데이터 폴더 경로
data_path = "data/population_reports/" 
pdf_files = glob(os.path.join(data_path, "*.pdf"))  

# 이미지 저장 경로
image_output_dir = os.path.join(data_path, 'images')  
os.makedirs(image_output_dir, exist_ok=True)

print(f"PDF files:")
pprint(pdf_files)

PDF files:
['data/population_reports/2025년_5월_경제활동인구조사_청년층_부가조사_결과.pdf',
 'data/population_reports/2024년_인구주택총조사_결과(등록센서스_방식).pdf']


In [7]:
for file_path in pdf_files:
    split_pdf_by_pages(file_path)

📄 원본 파일: 2025년_5월_경제활동인구조사_청년층_부가조사_결과.pdf
📊 총 페이지: 40
✂️ 분할 단위: 10페이지씩
✅ 생성: 2025년_5월_경제활동인구조사_청년층_부가조사_결과_chunk_001-010.pdf (페이지 1-10)
✅ 생성: 2025년_5월_경제활동인구조사_청년층_부가조사_결과_chunk_011-020.pdf (페이지 11-20)
✅ 생성: 2025년_5월_경제활동인구조사_청년층_부가조사_결과_chunk_021-030.pdf (페이지 21-30)
✅ 생성: 2025년_5월_경제활동인구조사_청년층_부가조사_결과_chunk_031-040.pdf (페이지 31-40)
🎉 분할 완료: 4개 파일

📄 원본 파일: 2024년_인구주택총조사_결과(등록센서스_방식).pdf
📊 총 페이지: 137
✂️ 분할 단위: 10페이지씩
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_001-010.pdf (페이지 1-10)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_011-020.pdf (페이지 11-20)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_021-030.pdf (페이지 21-30)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_031-040.pdf (페이지 31-40)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_041-050.pdf (페이지 41-50)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_051-060.pdf (페이지 51-60)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_061-070.pdf (페이지 61-70)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_071-080.pdf (페이지 71-80)
✅ 생성: 2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf (페이지 81-90)
✅ 생성: 2024년_인

### 2\) 문서 파싱

In [8]:
import os
import base64
from typing import List, Dict, Any
from io import BytesIO
import pandas as pd
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling.datamodel.document import TableItem
from docling.chunking import HybridChunker

# 파이프라인 옵션 설정
pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = 2.0 # 144 DPI (2.0 * 72)
pipeline_options.generate_page_images = True
pipeline_options.do_table_structure = True

# 문서 추출 옵션 설정
format_options = {
    InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
}

# 문서 추출 객체 생성
converter = DocumentConverter(format_options=format_options)

# Chunker 설정 (텍스트 청킹 활성화 시)
chunker = HybridChunker(
    tokenizer="BAAI/bge-m3",
    max_tokens=8000,
    merge_peers=True
)

def table_to_markdown(table: TableItem) -> str:
    """
    TableItem을 Markdown 형식으로 변환
    """
    # export_to_dataframe 메서드 사용
    if hasattr(table, 'export_to_dataframe'):
        df = table.export_to_dataframe()
        if df is not None and not df.empty:
            return df.to_markdown(index=False)

    # export_to_markdown 메서드 사용
    else:
        markdown = table.export_to_markdown()
        if markdown:
            return markdown

    return ""

def extract_text_chunks(doc, chunker=chunker) -> Dict[str, Any]:
    """
    텍스트 청크 추출
    """
    chunks = list(chunker.chunk(doc))
    text_chunks = []
    for idx, chunk in enumerate(chunks):
        chunk_data = {
            'index': idx,
            'text': chunk.text,
            'token_count': len(chunk.text.split()),  # 대략적인 토큰 수
            'metadata': {}
        }

        # 청크 메타데이터 추출
        if hasattr(chunk, 'meta') and chunk.meta:
            if hasattr(chunk.meta, 'page_range') and chunk.meta.page_range:
                chunk_data['metadata']['pages'] = list(chunk.meta.page_range)
    
        text_chunks.append(chunk_data)

    return text_chunks

def extract_tables(doc) -> Dict[str, Any]:
    """
    테이블 추출
    """
    tables = {}
    if not hasattr(doc, 'tables') or not doc.tables:
        return tables
    
    for idx, table in enumerate(doc.tables):
        if not isinstance(table, TableItem):
            continue
        
        markdown_table = table_to_markdown(table)
        
        if not markdown_table:
            print(f"    ⚠️ 테이블 {idx} 변환 결과가 비어있음")
            continue

        table_key = f"table_{idx}"
        page_no = 'unknown'
        if hasattr(table, 'prov') and table.prov:
            page_no = table.prov[0].page_no if hasattr(table.prov[0], 'page_no') else 'unknown'
            table_key = f"table_{idx}_page{page_no}"
        
        tables[table_key] = {
            'markdown': markdown_table,
            'page_no': page_no,
            'table_index': idx,
        }
        print(f"    ✅ 테이블 {idx} 변환 성공: {len(markdown_table)} 문자")

    return tables

def extract_page_images(doc) -> Dict[str, Any]:
    """
    페이지 이미지 추출
    """
    page_images = {}
    for page_no, page in doc.pages.items():
        try:
            if not hasattr(page, 'image') or page.image is None:
                print(f"    ⚠️ 페이지 {page_no}: 이미지 객체 없음")
                continue

            if not hasattr(page.image, 'pil_image') or page.image.pil_image is None:
                print(f"    ⚠️ 페이지 {page_no}: PIL 이미지 없음")
                continue

            # PIL 이미지를 base64로 변환
            pil_img = page.image.pil_image
            
            # PNG 형식으로 base64 인코딩
            buffered = BytesIO()
            pil_img.save(buffered, format="PNG")
            img_bytes = buffered.getvalue()
            img_base64 = base64.b64encode(img_bytes).decode('utf-8')
            
            page_images[f"page_{page_no}"] = {
                'base64': f"data:image/png;base64,{img_base64}",
                'width': pil_img.width,
                'height': pil_img.height,
                'page_no': page_no
            }
            print(f"    ✅ 페이지 {page_no}: {pil_img.width}x{pil_img.height} 이미지 추출 성공")
                
        except Exception as e:
            print(f"    ❌ 페이지 {page_no} 이미지 추출 실패: {e}")

    return page_images


def create_metadata(doc) -> Dict[str, Any]:
    """
    기본 메타데이터 수집
    """
    meta_data = {
        'pages': len(doc.pages),
        'pictures': len(doc.pictures) if hasattr(doc, 'pictures') else 0,
        'tables': len(doc.tables) if hasattr(doc, 'tables') else 0
    }
    return meta_data

In [9]:
import json
import os

def save_extracted_content(output_dir: str, document_content) -> None:
    """
    추출된 콘텐츠를 파일로 저장
    """ 
    os.makedirs(output_dir, exist_ok=True)

    file_name = os.path.splitext(os.path.basename(document_content["source"]))[0]
    metadata = {
        'file_info': document_content['metadata'],
    }
    
    # 전체 텍스트를 마크다운으로 저장
    if document_content['text_content']:
        md_path = os.path.join(output_dir, f"{file_name}_full_text.md")
        with open(md_path, 'w', encoding='utf-8') as f:
            f.write(document_content['text_content'])
        print(f"💾 텍스트 저장: {md_path}")
        metadata["text_length"] = len(document_content['text_content'])
    
    # 청크별 텍스트 저장
    if document_content['text_chunks']:
        chunks_dir = os.path.join(output_dir, f"{file_name}_chunks")
        os.makedirs(chunks_dir, exist_ok=True)
        
        for chunk in document_content['text_chunks']:
            chunk_path = os.path.join(chunks_dir, f"chunk_{chunk['index']:03d}.md")
            with open(chunk_path, 'w', encoding='utf-8') as f:
                f.write(f"# Chunk {chunk['index']}\n\n")
                f.write(f"**Token Count:** ~{chunk['token_count']}\n\n")
                if chunk['metadata'].get('pages'):
                    f.write(f"**Pages:** {chunk['metadata']['pages']}\n\n")
                f.write("---\n\n")
                f.write(chunk['text'])
        
        print(f"💾 청크 저장: {chunks_dir} ({len(document_content['text_chunks'])}개)")
        metadata["chunk_count"] = len(document_content['text_chunks'])
    
    # 테이블을 개별 마크다운으로 저장
    if document_content.get('tables'):
        tables_dir = os.path.join(output_dir, f"{file_name}_tables")
        os.makedirs(tables_dir, exist_ok=True)
        
        for table_key, table_info in document_content['tables'].items():
            table_path = os.path.join(tables_dir, f"{table_key}.md")
            with open(table_path, 'w', encoding='utf-8') as f:
                f.write(f"# {table_key}\n\n")
                f.write(f"**Page:** {table_info['page_no']}\n\n")
                f.write("---\n\n")
                f.write(table_info['markdown'])
        
        print(f"💾 테이블 저장: {tables_dir} ({len(document_content['tables'])}개)")
        metadata["table_count"] = len(document_content['tables'])
        metadata["tables_info"] = {k: {**v, 'markdown': None} for k, v in document_content['tables'].items()}
    
    # 페이지 이미지 저장
    if document_content.get('page_images'):
        images_dir = os.path.join(output_dir, f"{file_name}_images")
        os.makedirs(images_dir, exist_ok=True)
        
        for page_key, page_info in document_content['page_images'].items():
            # base64 데이터를 디코딩하여 파일로 저장
            base64_data = page_info['base64'].split(',')[1]
            img_bytes = base64.b64decode(base64_data)
            
            img_path = os.path.join(images_dir, f"{page_key}.png")
            with open(img_path, 'wb') as f:
                f.write(img_bytes)
        
        print(f"💾 이미지 저장: {images_dir} ({len(document_content['page_images'])}개)")
        metadata["image_count"] = len(document_content['page_images'])
        metadata["images_info"] = {k: {**v, 'base64': None} for k, v in document_content['page_images'].items()}
        
    # 메타데이터를 JSON으로 저장
    metadata_path = os.path.join(output_dir, f"{file_name}_metadata.json")
    with open(metadata_path, 'w', encoding='utf-8') as f:
        # base64 이미지 데이터는 제외하고 메타데이터만 저장
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"💾 메타데이터 저장: {metadata_path}")

In [10]:
from llama_parse import LlamaParse

# 문서 파서 초기화
user_prompt = """
Please extract data in a structured format that is suitable for demographic analysis. Focus on:
1. Tables with clear headers and consistent formatting
2. Numerical data with proper alignment and units
3. Time series data (years, dates) in chronological order
4. Statistical indicators like birth rates, death rates, fertility rates, etc.
5. Geographic divisions (cities, provinces, regions) clearly labeled
6. Preserve the relationship between data points and their corresponding metadata
7. Ensure demographic terminology and classifications are accurately maintained
"""
parser = LlamaParse(
    result_type="markdown",  # 마크다운 형식으로 출력
    verbose=True,            # 상세 로그 출력
    language="ko",           # 한국어 설정
    user_prompt=user_prompt, # 사용자 정의 프롬프트
    split_by_page=True,      # 페이지 단위로 파싱
    num_workers=4,
)

In [ ]:
# 분할된 데이터 폴더 경로
data_path = "data/population_reports/" 
split_files = glob(os.path.join(data_path, 'split_pdfs', "*.pdf"))

# 추출된 파일 저장 경로
output_dir = os.path.join(data_path, 'extracted_content')
os.makedirs(output_dir, exist_ok=True)

# 문서 파싱 17분 소요
all_documents = {}
for idx, file_path in enumerate(split_files):
    result = {
        'source': file_path,
        'parent_id': f"parent_{idx:03d}",
    }

    try:
        print(f"📄 처리 중: {os.path.basename(file_path)}")
        conv_res = converter.convert(file_path)
        doc = conv_res.document
        result['text_content'] = doc.export_to_markdown()
        result['text_chunks'] = extract_text_chunks(doc, chunker)
        result['tables'] = extract_tables(doc)
        result['page_images'] = extract_page_images(doc)
        result['metadata'] = create_metadata(doc)
        save_extracted_content(output_dir, result)
    except Exception as e:
        print(f"❌ 파일 처리 실패 {file_path}: {e}")

        try:
            print(f"LlamaParse 파싱 시도")
            doc = parser.load_data(file_path)

            combined_text = ""
            chunks = []
            for idx, page in enumerate(doc):
                chunks.append({
                    'index': idx,
                    'text': page.text,
                    'token_count': len(page.text.split()),
                    'metadata': page.metadata
                })
                combined_text += page.text + "\n\n"

            result['text_content'] = combined_text
            result['text_chunks'] = chunks
            result['metadata'] = {
                'pages': len(doc)
            }
            save_extracted_content(output_dir, result)
            print(f"✅ LlamaParse로 처리 완료: {file_path}")
        except Exception as e:
            print(f"❌ LlamaParse 파싱 실패 {file_path}: {e}")

    all_documents[file_path] = result

In [28]:
len(all_documents)

18

In [49]:
import pickle
import os

with open(os.path.join(data_path, 'all_documents.pkl'), 'wb') as f:
    pickle.dump(all_documents, f)

print(f"✅ all_documents 저장 완료: {len(all_documents)}개 문서")


✅ all_documents 저장 완료: 18개 문서


---

## 2. 문서 인덱싱

### 1\) 자식 문서 저장소 정의

In [39]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 요약 문서 저장
vectorstore = Chroma(
    collection_name="population_child",
    embedding_function=embeddings,
    persist_directory="./population_db"
)

### 2\) 이미지 내용 요약

In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_message = """
당신은 한국의 인구 동향과 통계를 전문적으로 분석하는 인구학 전문가입니다.

다음 이미지 문서의 내용을 정확하고 체계적으로 요약해 주세요:

**분석 지침:**
1. **텍스트 내용**: 핵심 통계 수치, 증감률, 비율 등을 정확히 기록
2. **표(Table) 분석**: 
   - 표의 제목, 단위, 구성 요소를 명확히 파악
   - 행과 열의 데이터를 체계적으로 정리
   - 주요 수치와 변화 추이를 강조
3. **차트/그래프 분석**:
   - 차트 유형(막대, 선, 원형 등) 명시
   - X축, Y축의 의미와 범위 설명
   - 주요 데이터 포인트와 트렌드 파악
   - 비교 대상과 변화 패턴 분석
4. **이미지 내 텍스트**: 범례, 주석, 설명문 등 모든 텍스트 정보 포함

**요약 형식:**
- 문서 유형과 주제를 먼저 명시
- 핵심 통계 수치를 구체적으로 나열
- 표나 차트의 주요 발견사항을 불릿 포인트로 정리
- 시계열 데이터의 경우 변화 추이와 패턴 강조
- 지역별, 연령별, 성별 등 세분화된 데이터 구체적으로 기술

**주의사항:**
- 모든 수치는 정확히 기록 (단위, 천 단위 구분자 포함)
- 추상적 표현보다는 구체적 데이터 중심으로 작성
- 원본 문서의 구조와 논리적 흐름 유지
- 검색 키워드가 될 수 있는 용어들을 포함하여 작성

"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", [{"type": "image_url", "image_url": {"url": "{base64}"}}])
])
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
summary_chain = prompt | llm | StrOutputParser()

In [ ]:
from langchain_core.documents import Document

# 이미지 자식 문서 생성 (요약) - 29분 소요
idx = 0
child_docs_images = []
for key, doc in all_documents.items():
    print(key)
    if not isinstance(doc, dict) or doc.get('page_images') is None:
        print(f"❌ {key} - 이미지 데이터 없음")
        continue

    parent_id = doc['parent_id']
    for page_num, page_image in doc['page_images'].items():
        child_id = f"child_images_{idx:03d}"
        summary = summary_chain.invoke(page_image)
        child_doc = Document(
            page_content=summary,
            metadata={
                "parent_id": parent_id, 
                "child_id": child_id, 
                "source": key, 
                "page_image": page_image.get("base64", "")
            }
        )

        child_docs_images.append(child_doc)
        idx += 1
        print(f"✅ {key}, {page_num} - 이미지 요약 완료 : {summary[:100]}")


data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, page_1 - 이미지 요약 완료 : 문서 유형 및 주제:
- 문서 유형: 통계표 및 설명문
- 주제: 2024년 건축연도 및 주택종류별 미거주 주택(빈집) 현황 분석

핵심 통계 수치:
- 1989년 이전 건축된 미
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, page_2 - 이미지 요약 완료 : 문서 유형 및 주제:
- 시도별 노후기간 30년 이상 된 미거주 주택(빈집) 현황 및 증감 분석 (2023~2024년)

핵심 통계 수치:
- 전국 30년 이상 된 미거주 주택 비
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, page_3 - 이미지 요약 완료 : - 문서 유형 및 주제:  
  2024년 시도별 미거주 주택(빈집) 비율 및 노후기간 30년 이상 된 미거주 주택(빈집) 비율 분석

- 핵심 통계 수치 (그림 54, 2024년
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, page_4 - 이미지 요약 완료 : 문서 유형: 2024년 (반)지하 및 옥탑 주택과 가구 현황 통계 보고서

주제: 2024년 11월 1일 기준 (반)지하 및 옥탑이 있는 주택과 이에 거주하는 가구 현황

---

✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, page_5 - 이미지 요약 완

In [48]:
# 이미지 요약 문서 저장
vectorstore.add_documents(child_docs_images)

['768bf4a9-ecce-4b06-a642-fd238ed0a417',
 '1d709b55-d7cb-4820-883f-41ef154c380a',
 '1b88649d-50fc-42a5-bf43-0c275c5c1e9f',
 'cf05ae75-dd83-4071-8320-88e5850471ef',
 'f96a3bb3-38ff-41a4-b78f-8a7f27510c3b',
 '20867525-b04f-47ad-b2a5-6d1e9a82a6c2',
 '7d704599-b0c5-4304-91d6-9919630f7cca',
 '8fb0e290-68ce-4ec7-9b4d-1db48705d54b',
 '106ff12d-0176-4475-bcfa-34eac741237d',
 'e3415614-4acc-47cf-97d7-cff7419c4dbd',
 '55011d60-98e0-41cb-bfc7-86c3b9f024c9',
 '4f33f8f4-166b-4d41-bb2f-0b64a0c3e364',
 'fdf37ae7-cce0-4a02-9d45-62fe5ade91dd',
 'bd006a03-f176-42bd-aefd-371b9c1f4e4c',
 '28ddf0ba-5386-4d74-8a41-4677aa953fe2',
 'a9815a9d-28ab-4aa2-843c-37d154e85a46',
 '36763bea-a542-4eb4-b4fd-5621f55bad25',
 '89b50ba6-865c-40bb-8e65-4dfd092161aa',
 '2978622a-6fdc-4729-ad33-b6255150dffa',
 '6fea8a37-dd03-4b56-a82e-514741b9311f',
 'e438434b-b231-44aa-8234-544cd2f8b3a4',
 'b7a9eb27-77cf-4edf-bad9-5e06ea487f1b',
 '4abd46b1-1c77-4b0e-b26b-a05cef040ad1',
 'f3e6cefe-c1b3-4b20-a141-90397c36ccd0',
 '0580fb6d-4b74-

### 3\) 테이블 자식 문서 저장

In [50]:
from langchain_core.documents import Document

# 테이블 자식 문서 생성
idx = 0
child_docs_tables = []
for key, doc in all_documents.items():
    print(key)
    if not isinstance(doc, dict) or doc.get('tables') is None:
        print(f"❌ {key} - 테이블 데이터 없음")
        continue

    parent_id = doc['parent_id']
    for table_num, table in doc['tables'].items():
        child_id = f"child_tables_{idx:03d}"
        child_doc = Document(
            page_content=table.get("markdown", ""),
            metadata={"parent_id": parent_id, "child_id": child_id, "source": key,}
        )

        child_docs_tables.append(child_doc)
        idx += 1
        print(f"✅ {key}, {table_num} - 테이블 문서 생성 완료")


data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_0_page1 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_1_page2 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_2_page4 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_3_page4 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_4_page5 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_5_page5 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_6_page6 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_7_page7 - 테이블 문서 생성 완료
✅ data/population_report

In [51]:
# 테이블 문서 저장
vectorstore.add_documents(child_docs_tables)

['ddc4c532-d3b9-4180-bc6c-ba5ca9a2da40',
 '043dc33b-600f-4740-98e7-d12ef5bcaf34',
 'f118d570-aff1-4b52-a994-6182e4628096',
 '9237bcaa-c584-4912-9bee-66205b84ebc9',
 '36265e1a-6577-4c02-8c7f-e5e35f7633e0',
 '36385a84-09ad-461b-a209-a836ffea8164',
 '3420e9c3-648e-42bd-be4d-86558ab66bde',
 'b8c2398e-3d1c-42b4-aac6-3661496fe940',
 '22147341-2002-4c52-b647-5c5c10eaaf1a',
 '682a72e0-fd0e-4953-aa1c-4725f89cebb1',
 '9771203c-ca28-4fc7-85cf-76cc8c9bfe8d',
 '7cd5a696-efb4-433f-9a8d-a655d4660185',
 '8536cb37-e508-4dd7-a391-09ab97e8ac7d',
 'a1518bbf-40fd-4f43-a637-4d80f37cd396',
 '2015f20c-7c55-41f6-9d69-ef3594d59e6f',
 '02453f1c-d891-453e-9db9-637c62e7dc2a',
 '7646058d-1b10-4b3f-b9a1-d5a76d65ba7f',
 '241009ec-06f1-4c56-8cfa-763fa9b76eca',
 '02c99ee8-914f-445d-b848-f0bf2172b49f',
 'bf11d622-6935-4f11-94ba-442c04bfc8af',
 '3a5862be-93b9-40e4-98bd-263d6449c8b1',
 '7eafd661-2a6e-4582-966c-d84e6be7f84a',
 '1323b5c7-7209-40ae-9db4-6e23042b0473',
 'f1475842-d33f-4970-bc93-13a0c190b359',
 '512254b4-9b66-

### 4\) 청크 자식 문서 저장

In [52]:
from langchain_core.documents import Document

# 테이블 자식 문서 생성
idx = 0
child_docs_chunks = []
for key, doc in all_documents.items():
    print(key)
    if not isinstance(doc, dict) or doc.get('text_chunks') is None:
        print(f"❌ {key} - 문서 데이터 없음")
        continue

    parent_id = doc['parent_id']
    for chunk_data in doc['text_chunks']:
        child_id = f"child_chunks_{idx:03d}"
        child_doc = Document(
            page_content=chunk_data.get("text", ""),
            metadata={"parent_id": parent_id, "child_id": child_id, "source": key,}
        )

        child_docs_chunks.append(child_doc)
        idx += 1
        print(f"✅ {key}, {table_num} - 테이블 문서 생성 완료")


data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/population_reports/split_pdfs/2024년_인구주택총조사_결과(등록센서스_방식)_chunk_081-090.pdf, table_10_page10 - 테이블 문서 생성 완료
✅ data/p

In [53]:
# 청크 문서 저장
vectorstore.add_documents(child_docs_chunks)

['cbcdb3c5-fe4c-4f17-bcbb-bf757a9cd313',
 '7b067d54-79c2-4039-b2ae-ee2ccb12b9a0',
 '09075b29-2552-4ab4-bcad-2e7b95aab305',
 'a2b27c5e-05a5-439b-badf-aa2d664c5b6d',
 'e98c1065-5086-4db9-94d2-10e701d8095d',
 '65c07237-5e52-4d31-b8ad-ca78f9e80dc2',
 '75690eff-1da6-4551-b13b-b655a8c08b5d',
 'cf6e7581-ac84-429e-8b1e-0360d1bc1064',
 'f97fe993-c63a-4bb4-8a49-89d186921c5f',
 'a5a5f362-5838-4c51-8d7a-ff64b74a179e',
 'bccadc3c-36fe-4cf4-ad6e-58ccf262db8d',
 '15719c4a-d818-469e-b5a7-2148e82922ce',
 '117d2871-a0a7-49ea-8da4-fddc77667921',
 'c8bcd15e-f399-4c4e-8a48-88f12271522a',
 'dfe7f113-8ad0-463a-adb8-c4be38318a02',
 '75467a03-838b-4657-8ae0-d39ce8a5ca17',
 '94d9046e-74ae-438e-892c-eb96957cf575',
 '0339fd8c-4e69-44b6-95bd-fdc713652916',
 'af2362bf-87eb-4466-b78a-b0a82680876e',
 '8461a21f-0c5f-4356-94ea-3d43bdcb205d',
 'a39386fb-3217-4c4a-80d2-c3bcd1eb73bf',
 'ea282ca9-bf6c-4124-bbc3-700d31e29c55',
 'cabab9c1-40a7-4ec1-8e43-d29e2046d929',
 'b77ec353-eb2f-40a0-b54f-6326bb5499ee',
 '707ba07a-65aa-

### 5\) 부모 문서 저장

In [54]:
from langchain.storage import InMemoryStore

parent_docs = []
for key, doc in all_documents.items():
    parent_id = doc['parent_id']
    parent_doc = Document(
        page_content=doc['text_content'],
        metadata={"parent_id": parent_id, "source": key}
    )
    parent_docs.append(parent_doc)
    
store = InMemoryStore()
store.mset([(doc.metadata["parent_id"], doc) for doc in parent_docs])

### 6\) 검색기 생성

In [68]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key="parent_id",             # child 문서 id 필드
    # parent_id_key="parent_id"      # child 문서에 있는 parent 문서 id 필드
)

## 3. 멀티 모달 RAG 체인 정의 및 테스트

In [89]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
import base64
from io import BytesIO
from PIL import Image

# LLM 초기화
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

# 시스템 메시지 정의
system_message = """당신은 인구 분석 전문가입니다. 사용자의 질의에 대해 제공된 참고 문서를 기반으로 정확하고 상세한 답변을 생성해주세요.

다음 지침을 따라주세요:
1. 제공된 참고 문서의 내용을 정확히 분석하고 인용하여 답변하세요.
2. 통계 수치나 데이터가 포함된 경우 정확한 수치를 명시하세요.
3. 이미지가 포함된 경우 이미지의 내용도 함께 분석하여 답변에 반영하세요.
4. 논리적이고 체계적으로 답변을 구성하세요.
5. 참고 문서에 없는 내용은 추측하지 말고, 문서 범위 내에서만 답변하세요.
6. 필요시 표나 차트의 내용을 텍스트로 설명하여 이해를 돕도록 하세요."""

def format_docs_with_images(docs):
    """문서를 포맷팅하고 이미지가 있으면 함께 처리"""
    formatted_docs = []
    image_contents = []
    
    for doc in docs:
        # 텍스트 내용 추가
        formatted_docs.append(f"출처: {doc.metadata.get('source', 'Unknown')}\n내용: {doc.page_content}")
        
        # 이미지가 있는지 확인 (child_id에 'images'가 포함되어 있으면)
        child_id = doc.metadata.get('child_id', '')
        if 'images' in child_id and hasattr(doc, 'metadata'):
            # 이미지 데이터가 있으면 추가
            if doc.metadata.get('page_image'):
                image_contents.append({
                    "type": "image_url",
                    "image_url": {
                        "url": doc.metadata['page_image']
                    }
                })
    
    return "\n\n".join(formatted_docs), image_contents

def create_multimodal_prompt(question, docs):
    """멀티모달 프롬프트 생성"""
    text_content, image_contents = format_docs_with_images(docs)
    
    # 프롬프트 메시지 구성
    messages = [
        {
            "role": "system",
            "content": system_message
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"질문: {question}\n\n참고 문서:\n{text_content}\n\n"
                }
            ] + image_contents
        }
    ]
    
    return messages

# RAG 체인 생성
def get_relevant_docs(question):
    """관련 문서 검색"""
    return retriever.get_relevant_documents(question, k=10)

def multimodal_rag_chain(question):
    """멀티모달 RAG 체인 실행"""
    # 1. 관련 문서 검색
    docs = get_relevant_docs(question)
    
    # 2. 멀티모달 프롬프트 생성
    messages = create_multimodal_prompt(question, docs)
    
    # 3. LLM 호출
    response = llm.invoke(messages, config={"callbacks": [langfuse_handler]})
    
    return {
        "question": question,
        "answer": response.content,
        "source_docs": docs
    }


In [90]:
testset = [
    "2024년 인구주택총조사에서 미거주 주택(빈집) 현황은 어떻게 나타났나요?",
    "건축연도가 1989년 이전인 빈집의 비율과 특징을 설명해주세요.",
    "2025년 5월 경제활동인구조사에서 청년층의 취업률은 어떻게 나타났나요?",
    "청년층의 고용 현황과 주택 문제 사이의 관련성을 분석해주세요.",
    "인구주택총조사와 경제활동인구조사 결과를 종합하여 청년층의 주거 안정성에 대해 설명해주세요."
]

In [91]:
response = multimodal_rag_chain(testset[0])
print(response)

{'question': '2024년 인구주택총조사에서 미거주 주택(빈집) 현황은 어떻게 나타났나요?', 'answer': '2024년 인구주택총조사에서 미거주 주택(빈집) 현황은 다음과 같이 나타났습니다.\n\n---\n\n## 1. 미거주 주택(빈집) 규모 및 비율\n\n- **2024년 11월 1일 기준 미거주 주택(빈집)은 1,599천 호**로, 전체 주택(19,873천 호)의 **8.0%**를 차지합니다.\n    - 전년(2023년) 대비 64천 호(4.2%) 증가, 비율은 0.2%p 상승(2023년 7.9% → 2024년 8.0%)\n    - 5년 전(2019년)과 비교하면 81천 호(5.4%) 증가, 비율은 0.3%p 감소(2019년 8.4% → 2024년 8.0%)\n\n| 연도   | 총주택(천 호) | 미거주주택(천 호) | 비율(%) |\n|--------|--------------|-------------------|---------|\n| 2019   | 18,127       | 1,518             | 8.4     |\n| 2020   | 18,526       | 1,511             | 8.2     |\n| 2021   | 18,812       | 1,395             | 7.4     |\n| 2022   | 19,156       | 1,452             | 7.6     |\n| 2023   | 19,546       | 1,535             | 7.9     |\n| 2024   | 19,873       | 1,599             | 8.0     |\n\n- **미거주 주택(빈집)의 정의**: 11월 1일 기준 사람이 살지 않는 주택을 의미하며, 신축주택 및 매매·이사 등으로 인한 일시적 미거주 주택도 포함합니다.\n\n---\n\n## 2. 건축연도 및 주택종류별 미거주 주택(빈집) 현황\n\n- **1989년 이전에 건축된 미거주 주택(빈집)은 4

In [92]:
response = multimodal_rag_chain(testset[1])
print(response)

{'question': '건축연도가 1989년 이전인 빈집의 비율과 특징을 설명해주세요.', 'answer': '아래는 2024년 인구주택총조사 결과(등록센서스 방식)를 바탕으로, **건축연도가 1989년 이전인 빈집(미거주 주택)의 비율과 특징**에 대한 상세한 설명입니다.\n\n---\n\n## 1. 1989년 이전 건축 빈집의 비율\n\n- **1989년 이전에 건축된 주택**: 3,067천 호\n- **이 중 미거주 주택(빈집)**: 468천 호\n- **비율**: 15.3%\n\n  > *즉, 1989년 이전에 지어진 주택의 15.3%가 현재 빈집(미거주 주택)입니다.*  \n  (출처: 표 51, chunk_081-090.pdf)\n\n---\n\n## 2. 1989년 이전 빈집의 주택종류별 특징\n\n| 주택종류                | 전체(천 호) | 빈집(천 호) | 빈집 비율(%) |\n|------------------------|-------------|-------------|--------------|\n| **단독주택**           | 1,786       | 301         | 16.8         |\n| **아파트**             | 899         | 104         | 11.6         |\n| **연립주택**           | 166         | 26          | 15.6         |\n| **다세대주택**         | 151         | 25          | 16.6         |\n| **비주거용 건물내주택** | 64          | 12          | 18.6         |\n\n- **단독주택**과 **다세대주택**, **비주거용 건물내주택**에서 빈집 비율이 16% 이상으로 높게 나타남.\n- **아파트**의 빈집 비율은 11.6%로 상대적으로 낮음.\n\n---\n\n## 3. 1989년 이전 빈집의 주요 특징\n

In [93]:
response = multimodal_rag_chain(testset[2])
print(response)

{'question': '2025년 5월 경제활동인구조사에서 청년층의 취업률은 어떻게 나타났나요?', 'answer': '2025년 5월 경제활동인구조사에서 청년층(15~29세)의 취업률(고용률)은 다음과 같이 나타났습니다.\n\n---\n\n### 1. 청년층 취업률(고용률) 주요 수치\n\n- **청년층(15~29세) 고용률:**  \n  **46.2%**  \n  (전년동월대비 0.7%p 하락)\n\n- **청년층 취업자 수:**  \n  **368만 2천 명**  \n  (전년동월대비 15만 명 감소)\n\n- **청년층 경제활동참가율:**  \n  **49.5%**  \n  (전년동월대비 0.8%p 하락)\n\n- **청년층 실업률:**  \n  **6.6%**  \n  (전년동월대비 0.1%p 하락)\n\n---\n\n### 2. 연령대별 고용률\n\n- **15~19세:** 6.3% (전년동월대비 1.1%p 하락)\n- **20~24세:** 45.8% (전년동월대비 0.7%p 하락)\n- **25~29세:** 72.7% (전년동월과 동일)\n\n---\n\n### 3. 표로 정리한 주요 지표  \n(단위: 천명, %, %p)\n\n| 구분         | 2024년 5월 | 2025년 5월 | 증감    |\n|--------------|------------|------------|---------|\n| 청년층 인구  | 8,173      | 7,974      | -200    |\n| 취업자       | 3,832      | 3,682      | -150    |\n| 고용률       | 46.9%      | 46.2%      | -0.7%p  |\n| 실업자       | 276        | 262        | -14     |\n| 실업률       | 6.7%       | 6.6%       | -0.1%p  |\n\n---\n\n### 4. 해설 및 분석\n\n- 2025년 5월 기준 청년층(15~29세)의 고

In [94]:
response = multimodal_rag_chain(testset[3])
print(response)

{'question': '청년층의 고용 현황과 주택 문제 사이의 관련성을 분석해주세요.', 'answer': '청년층의 고용 현황과 주택 문제의 관련성 분석\n\n### 1. 청년층 고용 현황\n\n2025년 5월 기준, 청년층(15~29세) 인구는 797만 4천 명으로 전년 동월 대비 20만 명 감소하였으며, 청년층 경제활동참가율은 49.5%로 0.8%p 하락, 고용률은 46.2%로 0.7%p 하락하였습니다. 청년층 취업자는 368만 2천 명(전년 대비 15만 명 감소), 실업자는 26만 2천 명(1만 4천 명 감소), 실업률은 6.6%입니다.  \n- 연령별 고용률은 15~19세 6.3%, 20~24세 45.8%, 25~29세 72.7%로, 20대 초반의 고용률이 낮은 편입니다.\n- 졸업 후 첫 일자리가 임금근로자인 경우, 첫 취업까지 평균 11.3개월이 소요되고, 첫 일자리 평균 근속기간은 1년 6.4개월에 불과합니다.\n- 첫 일자리를 그만둔 주요 사유는 ‘보수, 근로시간 등 근로여건 불만족’이 46.4%로 가장 높습니다.\n\n### 2. 주택 문제 현황\n\n#### (1) 주택 노후화 및 주택 유형\n- 2024년 기준 전체 주택의 54.9%(1,090만 8천 호)가 20년 이상 된 노후주택이며, 30년 이상 된 주택도 28.0%(557만 4천 호)에 달합니다.\n- 아파트는 전체 주택의 65.3%(1,297만 4천 호)를 차지하나, 이 중 20년 이상 된 아파트가 48.4%(628만 호), 30년 이상 된 아파트도 19.4%(251만 7천 호)입니다.\n- 주거용 연면적이 60~100㎡ 이하인 주택이 42.8%로 가장 많고, 40~60㎡ 이하가 28.3%, 40㎡ 이하 소형주택도 13.0%를 차지합니다.\n\n#### (2) 주택당 거주인수 및 미거주 주택(빈집)\n- 2024년 주택당 평균 거주인수는 2.6명으로 5년 전 대비 0.3명 감소했습니다. 아파트는 2.5명, 연립주택 2.2명, 다세대주택 2.1명으로 1~2인 가구 비중이 높아지고 

In [95]:
response = multimodal_rag_chain(testset[4])
print(response)

{'question': '인구주택총조사와 경제활동인구조사 결과를 종합하여 청년층의 주거 안정성에 대해 설명해주세요.', 'answer': '인구주택총조사와 경제활동인구조사 결과를 종합하여 청년층(15~29세)의 주거 안정성에 대해 분석하면 다음과 같습니다.\n\n---\n\n## 1. 청년층 인구 및 경제활동 현황\n\n- **청년층 인구**는 2025년 5월 기준 797만 4천 명으로, 전체 15세 이상 인구(4,573만 4천 명)의 17.4%를 차지합니다. 전년 동월 대비 20만 명 감소하였습니다.\n- **경제활동참가율**은 49.5%로 전년 대비 0.8%p 하락, **고용률**은 46.2%로 0.7%p 하락하였습니다.\n- **취업자 수**는 368만 2천 명(전년 대비 15만 명 감소), **실업자 수**는 26만 2천 명(1만 4천 명 감소)입니다.\n- **비경제활동인구**는 403만 명으로, 이 중 14.5%가 취업시험을 준비하고 있습니다(전년 대비 0.6%p 상승).\n\n---\n\n## 2. 청년층의 학업 및 취업 준비 상태\n\n- **최종학교 졸업자**는 417만 5천 명(52.4%), **재학생**은 347만 명(43.5%)입니다.\n- **졸업 후 취업경험자 비율**은 86.4%로, 취업경험이 한 번인 경우가 43.2%입니다.\n- **졸업 후 첫 일자리 취업까지 평균 소요기간**은 11.3개월, **첫 일자리 평균 근속기간**은 1년 6.4개월입니다.\n- **첫 일자리 임금**은 200~300만 원 미만이 39.7%로 가장 많고, 150~200만 원 미만이 28.3%, 100~150만 원 미만이 11.1%입니다.\n- **첫 일자리를 그만둔 사유**로는 ‘보수, 근로시간 등 근로여건 불만족’이 46.4%로 가장 높습니다.\n\n---\n\n## 3. 청년층의 주거 안정성 시사점\n\n### 1) 경제적 기반의 불안정\n\n- **고용률 하락**(46.2%, -0.7%p)과 **취업자 수 감소**(15만 명 감소)는 청년층의 소득 